In [1]:
import csv
import glob, pickle
import IPython.display
import librosa
import librosa.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tqdm import tqdm

2023-04-13 12:59:56.151255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
annotation_folder_route = './KEMDy20/annotation/'
wav_folder_route = './KEMDy20/wav/'

annotation_csv_list = os.listdir(annotation_folder_route)
session_list = os.listdir(wav_folder_route)

if '.DS_Store' in annotation_csv_list:
    annotation_csv_list.remove('.DS_Store')
    
if '.DS_Store' in session_list:
    session_list.remove('.DS_Store')

if '.ipynb_checkpoints' in annotation_csv_list:
    annotation_csv_list.remove('.ipynb_checkpoints')
    
if '.ipynb_checkpoints' in session_list:
    session_list.remove('.ipynb_checkpoints')

annotation_csv_list = sorted(annotation_csv_list)
session_list = sorted(session_list)

In [3]:
file_paths = []
labels = []

for i, annotation_csv in enumerate(annotation_csv_list):
    annotation_csv_file_route = annotation_folder_route + annotation_csv

    df = pd.read_csv(annotation_csv_file_route)
    df = df[['Segment ID', 'Total Evaluation']]
    df.drop([0], axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['Segment ID'] = df['Segment ID'] + '.wav'
    df.rename(columns={'Segment ID': 'file_name', 'Total Evaluation': 'emotion'}, inplace=True)
    
    label_list = df['emotion'].tolist()
    for j, label in enumerate(label_list):
        if len(label.split(';')) != 1:
            label_list[j] = label.split(';')[0]
    df['emotion'] = label_list
    
    wav_file_list = df['file_name'].tolist()
    
    session = session_list[i]
    
    for label in label_list:
        labels.append(label)
    
    for wav_file in wav_file_list:
        wav_file_route = wav_folder_route + session + '/' + wav_file
        file_paths.append(wav_file_route)

In [4]:
def extract_mfcc13(file_path):
    signal, sample_rate = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=13) # 13개의 MFCC 특징 추출
    # mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

def extract_mfcc22(file_path):
    signal, sample_rate = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=22) # 22개의 MFCC 특징 추출
    # mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

def extract_mfcc31(file_path):
    signal, sample_rate = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=31) # 31개의 MFCC 특징 추출
    # mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

def extract_mfcc40(file_path):
    signal, sample_rate = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40) # 40개의 MFCC 특징 추출
    # mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

In [5]:
mfccs13 = []

for file_path in file_paths:
    mfcc = extract_mfcc13(file_path)
    mfccs13.append(mfcc)

len(mfccs13)

13462

In [5]:
mfccs22 = []

for file_path in file_paths:
    mfcc = extract_mfcc22(file_path)
    mfccs22.append(mfcc)

len(mfccs22)

13462

In [5]:
mfccs31 = []

for file_path in file_paths:
    mfcc = extract_mfcc31(file_path)
    mfccs31.append(mfcc)

len(mfccs31)

13462

In [5]:
mfccs40 = []

for file_path in file_paths:
    mfcc = extract_mfcc40(file_path)
    mfccs40.append(mfcc)

len(mfccs40)

13462

In [7]:
## case 1. Zero-Padding 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs13)

# Zero Padding 적용
padded_data13 = []
for mfcc in mfccs13:
    pad_width = ((0, 0), (0, max_width - mfcc.shape[1]))
    padded_mfcc = np.pad(mfcc, pad_width=pad_width, mode='constant', constant_values=0)
    padded_data13.append(padded_mfcc)

In [7]:
## case 1. Zero-Padding 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs22)

# Zero Padding 적용
padded_data22 = []
for mfcc in mfccs22:
    pad_width = ((0, 0), (0, max_width - mfcc.shape[1]))
    padded_mfcc = np.pad(mfcc, pad_width=pad_width, mode='constant', constant_values=0)
    padded_data22.append(padded_mfcc)

In [6]:
## case 1. Zero-Padding 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs31)

# Zero Padding 적용
padded_data31 = []
for mfcc in mfccs31:
    pad_width = ((0, 0), (0, max_width - mfcc.shape[1]))
    padded_mfcc = np.pad(mfcc, pad_width=pad_width, mode='constant', constant_values=0)
    padded_data31.append(padded_mfcc)

In [6]:
## case 1. Zero-Padding 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs40)

# Zero Padding 적용
padded_data40 = []
for mfcc in mfccs40:
    pad_width = ((0, 0), (0, max_width - mfcc.shape[1]))
    padded_mfcc = np.pad(mfcc, pad_width=pad_width, mode='constant', constant_values=0)
    padded_data40.append(padded_mfcc)

In [8]:
## case 2. Time-Stretching 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs13)

# librosa.effects.time_stretch 함수를 사용하여 MFCC 배열의 길이를 조절
stretched_data13 = []
for mfcc in mfccs13:
    rate = max_width / mfcc.shape[1] # Time stretching 비율 계산
    stretched_mfcc = librosa.effects.time_stretch(mfcc, rate=rate)
    stretched_data13.append(stretched_mfcc)

# librosa.util.fix_length 함수를 사용하여 MFCC 배열의 길이를 최종적으로 동일하게 맞춤
stretched_data13 = [librosa.util.fix_length(mfcc, size=max_width) for mfcc in stretched_data13]

/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=348
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=504
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=355
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=492
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=90
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=372
  warnings.warn(

In [17]:
## case 2. Time-Stretching 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs22)

# librosa.effects.time_stretch 함수를 사용하여 MFCC 배열의 길이를 조절
stretched_data22 = []
for mfcc in mfccs22:
    rate = max_width / mfcc.shape[1] # Time stretching 비율 계산
    stretched_mfcc = librosa.effects.time_stretch(mfcc, rate=rate)
    stretched_data22.append(stretched_mfcc)

# librosa.util.fix_length 함수를 사용하여 MFCC 배열의 길이를 최종적으로 동일하게 맞춤
stretched_data22 = [librosa.util.fix_length(mfcc, size=max_width) for mfcc in stretched_data22]

/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=348
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=504
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=355
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=492
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=90
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=372
  warnings.warn(

In [7]:
## case 2. Time-Stretching 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs31)

# librosa.effects.time_stretch 함수를 사용하여 MFCC 배열의 길이를 조절
stretched_data31 = []
for mfcc in mfccs31:
    rate = max_width / mfcc.shape[1] # Time stretching 비율 계산
    stretched_mfcc = librosa.effects.time_stretch(mfcc, rate=rate)
    stretched_data31.append(stretched_mfcc)

# librosa.util.fix_length 함수를 사용하여 MFCC 배열의 길이를 최종적으로 동일하게 맞춤
stretched_data31 = [librosa.util.fix_length(mfcc, size=max_width) for mfcc in stretched_data31]

/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=348
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=504
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=355
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=492
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=90
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=372
  warnings.warn(

In [16]:
## case 2. Time-Stretching 과정

# 가장 긴 MFCC의 width 계산
max_width = max(mfcc.shape[1] for mfcc in mfccs40)

# librosa.effects.time_stretch 함수를 사용하여 MFCC 배열의 길이를 조절
stretched_data40 = []
for mfcc in mfccs40:
    rate = max_width / mfcc.shape[1] # Time stretching 비율 계산
    stretched_mfcc = librosa.effects.time_stretch(mfcc, rate=rate)
    stretched_data40.append(stretched_mfcc)

# librosa.util.fix_length 함수를 사용하여 MFCC 배열의 길이를 최종적으로 동일하게 맞춤
stretched_data40 = [librosa.util.fix_length(mfcc, size=max_width) for mfcc in stretched_data40]

/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=348
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=504
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=355
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=492
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=90
  warnings.warn(
/opt/anaconda3/envs/AI/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=372
  warnings.warn(

In [9]:
X_zero_13 = np.array(padded_data13) # X = mfccs
# y_zero_13 = np.array(labels)

In [8]:
X_zero_22 = np.array(padded_data22) # X = mfccs
# y_zero_22 = np.array(labels)

In [8]:
X_zero_31 = np.array(padded_data31) # X = mfccs
# y_zero_31 = np.array(labels)

In [7]:
X_zero_40 = np.array(padded_data40) # X = mfccs
# y_zero_40 = np.array(labels)

In [10]:
X_time_13 = np.array(stretched_data13) # X = mfccs
# y_time_13 = np.array(labels)

In [18]:
X_time_22 = np.array(stretched_data22) # X = mfccs
# y_time_22 = np.array(labels)

In [9]:
X_time_31 = np.array(stretched_data31) # X = mfccs
# y_time_31 = np.array(labels)

In [17]:
X_time_40 = np.array(stretched_data40) # X = mfccs
# y_time_40 = np.array(labels)

In [10]:
y = np.array(labels)

In [12]:
## 정규화

# X 배열을 2차원으로 reshape
X_zero_13_2d = X_zero_13.reshape(-1, X_zero_13.shape[-1])

# StandardScaler로 정규화
scaler_zero_13 = StandardScaler()
X_zero_13_2d_scaled = scaler_zero_13.fit_transform(X_zero_13_2d)

# 3차원으로 reshape
X_zero_13_scaled = X_zero_13_2d_scaled.reshape(X_zero_13.shape)

In [10]:
## 정규화

# X 배열을 2차원으로 reshape
X_zero_22_2d = X_zero_22.reshape(-1, X_zero_22.shape[-1])

# StandardScaler로 정규화
scaler_zero_22 = StandardScaler()
X_zero_22_2d_scaled = scaler_zero_22.fit_transform(X_zero_22_2d)

# 3차원으로 reshape
X_zero_22_scaled = X_zero_22_2d_scaled.reshape(X_zero_22.shape)

In [11]:
## 정규화

# X 배열을 2차원으로 reshape
X_zero_31_2d = X_zero_31.reshape(-1, X_zero_31.shape[-1])

# StandardScaler로 정규화
scaler_zero_31 = StandardScaler()
X_zero_31_2d_scaled = scaler_zero_31.fit_transform(X_zero_31_2d)

# 3차원으로 reshape
X_zero_31_scaled = X_zero_31_2d_scaled.reshape(X_zero_31.shape)

In [9]:
## 정규화

# X 배열을 2차원으로 reshape
X_zero_40_2d = X_zero_40.reshape(-1, X_zero_40.shape[-1])

# StandardScaler로 정규화
scaler_zero_40 = StandardScaler()
X_zero_40_2d_scaled = scaler_zero_40.fit_transform(X_zero_40_2d)

# 3차원으로 reshape
X_zero_40_scaled = X_zero_40_2d_scaled.reshape(X_zero_40.shape)

In [13]:
## 정규화

# X 배열을 2차원으로 reshape
X_time_13_2d = X_time_13.reshape(-1, X_time_13.shape[-1])

# StandardScaler로 정규화
scaler_time_13 = StandardScaler()
X_time_13_2d_scaled = scaler_time_13.fit_transform(X_time_13_2d)

# 3차원으로 reshape
X_time_13_scaled = X_time_13_2d_scaled.reshape(X_time_13.shape)

In [20]:
## 정규화

# X 배열을 2차원으로 reshape
X_time_22_2d = X_time_22.reshape(-1, X_time_22.shape[-1])

# StandardScaler로 정규화
scaler_time_22 = StandardScaler()
X_time_22_2d_scaled = scaler_time_22.fit_transform(X_time_22_2d)

# 3차원으로 reshape
X_time_22_scaled = X_time_22_2d_scaled.reshape(X_time_22.shape)

In [12]:
## 정규화

# X 배열을 2차원으로 reshape
X_time_31_2d = X_time_31.reshape(-1, X_time_31.shape[-1])

# StandardScaler로 정규화
scaler_time_31 = StandardScaler()
X_time_31_2d_scaled = scaler_time_31.fit_transform(X_time_31_2d)

# 3차원으로 reshape
X_time_31_scaled = X_time_31_2d_scaled.reshape(X_time_31.shape)

In [19]:
## 정규화

# X 배열을 2차원으로 reshape
X_time_40_2d = X_time_40.reshape(-1, X_time_40.shape[-1])

# StandardScaler로 정규화
scaler_time_40 = StandardScaler()
X_time_40_2d_scaled = scaler_time_40.fit_transform(X_time_40_2d)

# 3차원으로 reshape
X_time_40_scaled = X_time_40_2d_scaled.reshape(X_time_40.shape)

In [13]:
## 클래스 레이블 One-Hot Encoding

LE_train = LabelEncoder()
y_int = LE_train.fit_transform(y)
y = to_categorical(y_int, num_classes=7)

In [15]:
## train, test split
X_zero_13_train, X_zero_13_test, y_zero_13_train, y_zero_13_test = train_test_split(X_zero_13_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [12]:
## train, test split
X_zero_22_train, X_zero_22_test, y_zero_22_train, y_zero_22_test = train_test_split(X_zero_22_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [14]:
## train, test split
X_zero_31_train, X_zero_31_test, y_zero_31_train, y_zero_31_test = train_test_split(X_zero_31_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [11]:
## train, test split
X_zero_40_train, X_zero_40_test, y_zero_40_train, y_zero_40_test = train_test_split(X_zero_40_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [16]:
## train, test split
X_time_13_train, X_time_13_test, y_time_13_train, y_time_13_test = train_test_split(X_time_13_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [27]:
## train, test split
X_time_22_train, X_time_22_test, y_time_22_train, y_time_22_test = train_test_split(X_time_22_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [15]:
## train, test split
X_time_31_train, X_time_31_test, y_time_31_train, y_time_31_test = train_test_split(X_time_31_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [21]:
## train, test split
X_time_40_train, X_time_40_test, y_time_40_train, y_time_40_test = train_test_split(X_time_40_scaled, 
                                                                                    y, test_size=0.2, 
                                                                                    stratify=y, random_state=42)

In [17]:
X_zero_13_train.shape

(10769, 13, 1517)

In [13]:
X_zero_22_train.shape

(10769, 22, 1517)

In [16]:
X_zero_31_train.shape

(10769, 31, 1517)

In [12]:
X_zero_40_train.shape

(10769, 40, 1517)

In [17]:
X_time_13_train.shape

NameError: name 'X_time_13_train' is not defined

In [22]:
X_time_22_train.shape

(10769, 22, 1517)

In [17]:
X_time_31_train.shape

(10769, 31, 1517)

In [22]:
X_time_40_train.shape

(10769, 40, 1517)

In [47]:
np.random.seed(42)

# 모델을 정의합니다.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(13, 1517, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
np.random.seed(42)

# 모델을 정의합니다.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(22, 1517, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
np.random.seed(42)

# 모델을 정의합니다.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(31, 1517, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-04-12 11:37:16.500846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
np.random.seed(42)

# 모델을 정의합니다.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(40, 1517, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# 학습 수행 Zero-Padding, n=13
history_zero_13 = model.fit(X_zero_13_train, y_zero_13_train, 
                            validation_data=(X_zero_13_test, y_zero_13_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 41s 116ms/step - loss: 0.6368 - accuracy: 0.8333 - val_loss: 0.5853 - val_accuracy: 0.8355
Epoch 2/10
337/337 [==============================] - 39s 116ms/step - loss: 0.5794 - accuracy: 0.8365 - val_loss: 0.5792 - val_accuracy: 0.8366
Epoch 3/10
337/337 [==============================] - 39s 116ms/step - loss: 0.5407 - accuracy: 0.8428 - val_loss: 0.6069 - val_accuracy: 0.8273
Epoch 4/10
337/337 [==============================] - 39s 116ms/step - loss: 0.5132 - accuracy: 0.8482 - val_loss: 0.5914 - val_accuracy: 0.8396
Epoch 5/10
337/337 [==============================] - 39s 115ms/step - loss: 0.4737 - accuracy: 0.8537 - val_loss: 0.6035 - val_accuracy: 0.8440
Epoch 6/10
337/337 [==============================] - 39s 116ms/step - loss: 0.4399 - accuracy: 0.8641 - val_loss: 0.6599 - val_accuracy: 0.8362
Epoch 7/10
337/337 [==============================] - 39s 116ms/step - loss: 0.3996 - accuracy: 0.8732 - val_loss: 0.7214 - val_ac

In [15]:
# 학습 수행 Zero-Padding, n=22
history_zero_22 = model.fit(X_zero_22_train, y_zero_22_train, 
                            validation_data=(X_zero_22_test, y_zero_22_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 98s 286ms/step - loss: 0.6378 - accuracy: 0.8324 - val_loss: 0.5642 - val_accuracy: 0.8392
Epoch 2/10
337/337 [==============================] - 94s 280ms/step - loss: 0.5440 - accuracy: 0.8483 - val_loss: 0.5421 - val_accuracy: 0.8522
Epoch 3/10
337/337 [==============================] - 94s 279ms/step - loss: 0.4938 - accuracy: 0.8570 - val_loss: 0.5787 - val_accuracy: 0.8455
Epoch 4/10
337/337 [==============================] - 95s 281ms/step - loss: 0.4295 - accuracy: 0.8728 - val_loss: 0.5986 - val_accuracy: 0.8578
Epoch 5/10
337/337 [==============================] - 95s 281ms/step - loss: 0.3558 - accuracy: 0.8907 - val_loss: 0.6542 - val_accuracy: 0.8507
Epoch 6/10
337/337 [==============================] - 94s 280ms/step - loss: 0.2791 - accuracy: 0.9129 - val_loss: 0.8949 - val_accuracy: 0.8437
Epoch 7/10
337/337 [==============================] - 95s 282ms/step - loss: 0.2134 - accuracy: 0.9311 - val_loss: 0.9224 - val_ac

In [21]:
# 학습 수행 Zero-Padding, n=31
history_zero_31 = model.fit(X_zero_31_train, y_zero_31_train, 
                            validation_data=(X_zero_31_test, y_zero_31_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 165s 489ms/step - loss: 0.5836 - accuracy: 0.8381 - val_loss: 0.6216 - val_accuracy: 0.8288
Epoch 2/10
337/337 [==============================] - 165s 490ms/step - loss: 0.3843 - accuracy: 0.8802 - val_loss: 0.7371 - val_accuracy: 0.8407
Epoch 3/10
337/337 [==============================] - 164s 487ms/step - loss: 0.2604 - accuracy: 0.9182 - val_loss: 0.8547 - val_accuracy: 0.8362
Epoch 4/10
337/337 [==============================] - 157s 465ms/step - loss: 0.1679 - accuracy: 0.9477 - val_loss: 1.1016 - val_accuracy: 0.8284
Epoch 5/10
337/337 [==============================] - 155s 458ms/step - loss: 0.1025 - accuracy: 0.9683 - val_loss: 1.3696 - val_accuracy: 0.8147
Epoch 6/10
337/337 [==============================] - 154s 457ms/step - loss: 0.0717 - accuracy: 0.9785 - val_loss: 1.5402 - val_accuracy: 0.8177
Epoch 7/10
337/337 [==============================] - 154s 457ms/step - loss: 0.0396 - accuracy: 0.9900 - val_loss: 1.8025 -

In [14]:
# 학습 수행 Zero-Padding, n=40
history_zero_40 = model.fit(X_zero_40_train, y_zero_40_train, 
                            validation_data=(X_zero_40_test, y_zero_40_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 233s 688ms/step - loss: 0.6672 - accuracy: 0.8336 - val_loss: 0.5739 - val_accuracy: 0.8370
Epoch 2/10
337/337 [==============================] - 229s 679ms/step - loss: 0.5382 - accuracy: 0.8481 - val_loss: 0.5621 - val_accuracy: 0.8474
Epoch 3/10
337/337 [==============================] - 225s 669ms/step - loss: 0.4670 - accuracy: 0.8660 - val_loss: 0.5796 - val_accuracy: 0.8544
Epoch 4/10
337/337 [==============================] - 216s 641ms/step - loss: 0.3803 - accuracy: 0.8847 - val_loss: 0.6189 - val_accuracy: 0.8474
Epoch 5/10
337/337 [==============================] - 212s 630ms/step - loss: 0.2798 - accuracy: 0.9109 - val_loss: 0.7840 - val_accuracy: 0.8530
Epoch 6/10
337/337 [==============================] - 213s 633ms/step - loss: 0.1828 - accuracy: 0.9412 - val_loss: 0.9484 - val_accuracy: 0.8556
Epoch 7/10
337/337 [==============================] - 212s 628ms/step - loss: 0.1163 - accuracy: 0.9618 - val_loss: 1.1554 -

In [28]:
# 학습 수행 Time-Stretching, n=13
history_time_13 = model.fit(X_time_13_train, y_time_13_train, 
                            validation_data=(X_time_13_test, y_time_13_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 39s 116ms/step - loss: 0.6449 - accuracy: 0.8327 - val_loss: 0.6143 - val_accuracy: 0.8355
Epoch 2/10
337/337 [==============================] - 39s 116ms/step - loss: 0.6159 - accuracy: 0.8357 - val_loss: 0.6310 - val_accuracy: 0.8355
Epoch 3/10
337/337 [==============================] - 39s 115ms/step - loss: 0.6022 - accuracy: 0.8358 - val_loss: 0.6225 - val_accuracy: 0.8351
Epoch 4/10
337/337 [==============================] - 39s 115ms/step - loss: 0.5904 - accuracy: 0.8363 - val_loss: 0.6296 - val_accuracy: 0.8340
Epoch 5/10
337/337 [==============================] - 39s 115ms/step - loss: 0.5782 - accuracy: 0.8355 - val_loss: 0.6467 - val_accuracy: 0.8344
Epoch 6/10
337/337 [==============================] - 39s 115ms/step - loss: 0.5609 - accuracy: 0.8372 - val_loss: 0.6468 - val_accuracy: 0.8329
Epoch 7/10
337/337 [==============================] - 39s 115ms/step - loss: 0.5353 - accuracy: 0.8438 - val_loss: 0.6860 - val_ac

In [29]:
# 학습 수행 Time-Stretching, n=22
history_time_22 = model.fit(X_time_22_train, y_time_22_train, 
                            validation_data=(X_time_22_test, y_time_22_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 97s 286ms/step - loss: 0.6646 - accuracy: 0.8337 - val_loss: 0.6283 - val_accuracy: 0.8351
Epoch 2/10
337/337 [==============================] - 94s 279ms/step - loss: 0.6148 - accuracy: 0.8357 - val_loss: 0.6249 - val_accuracy: 0.8351
Epoch 3/10
337/337 [==============================] - 94s 279ms/step - loss: 0.5960 - accuracy: 0.8355 - val_loss: 0.6283 - val_accuracy: 0.8333
Epoch 4/10
337/337 [==============================] - 95s 282ms/step - loss: 0.5752 - accuracy: 0.8381 - val_loss: 0.6966 - val_accuracy: 0.8288
Epoch 5/10
337/337 [==============================] - 94s 279ms/step - loss: 0.5448 - accuracy: 0.8437 - val_loss: 0.7225 - val_accuracy: 0.8322
Epoch 6/10
337/337 [==============================] - 95s 283ms/step - loss: 0.5220 - accuracy: 0.8488 - val_loss: 0.7218 - val_accuracy: 0.8270
Epoch 7/10
337/337 [==============================] - 95s 282ms/step - loss: 0.4795 - accuracy: 0.8572 - val_loss: 0.7834 - val_ac

In [19]:
# 학습 수행 Time-Stretching, n=31
history_time_31 = model.fit(X_time_31_train, y_time_31_train, 
                            validation_data=(X_time_31_test, y_time_31_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 164s 485ms/step - loss: 0.6669 - accuracy: 0.8329 - val_loss: 0.6428 - val_accuracy: 0.8351
Epoch 2/10
337/337 [==============================] - 154s 458ms/step - loss: 0.6133 - accuracy: 0.8353 - val_loss: 0.6262 - val_accuracy: 0.8340
Epoch 3/10
337/337 [==============================] - 155s 460ms/step - loss: 0.5845 - accuracy: 0.8374 - val_loss: 0.6638 - val_accuracy: 0.8333
Epoch 4/10
337/337 [==============================] - 154s 458ms/step - loss: 0.5478 - accuracy: 0.8417 - val_loss: 0.6856 - val_accuracy: 0.8262
Epoch 5/10
337/337 [==============================] - 154s 458ms/step - loss: 0.5038 - accuracy: 0.8530 - val_loss: 0.7751 - val_accuracy: 0.8151
Epoch 6/10
337/337 [==============================] - 154s 456ms/step - loss: 0.4574 - accuracy: 0.8634 - val_loss: 0.8782 - val_accuracy: 0.8192
Epoch 7/10
337/337 [==============================] - 154s 456ms/step - loss: 0.4229 - accuracy: 0.8733 - val_loss: 0.9040 -

In [24]:
# 학습 수행 Time-Stretching, n=40
history_time_40 = model.fit(X_time_40_train, y_time_40_train, 
                            validation_data=(X_time_40_test, y_time_40_test), epochs=10, batch_size=32)

Epoch 1/10
337/337 [==============================] - 220s 651ms/step - loss: 0.6506 - accuracy: 0.8328 - val_loss: 0.6244 - val_accuracy: 0.8355
Epoch 2/10
337/337 [==============================] - 216s 640ms/step - loss: 0.6104 - accuracy: 0.8353 - val_loss: 0.6492 - val_accuracy: 0.8355
Epoch 3/10
337/337 [==============================] - 215s 636ms/step - loss: 0.5762 - accuracy: 0.8395 - val_loss: 0.6471 - val_accuracy: 0.8303
Epoch 4/10
337/337 [==============================] - 214s 636ms/step - loss: 0.5349 - accuracy: 0.8458 - val_loss: 0.7347 - val_accuracy: 0.8117
Epoch 5/10
337/337 [==============================] - 215s 637ms/step - loss: 0.4873 - accuracy: 0.8583 - val_loss: 0.7827 - val_accuracy: 0.8262
Epoch 6/10
337/337 [==============================] - 213s 632ms/step - loss: 0.4329 - accuracy: 0.8691 - val_loss: 0.9666 - val_accuracy: 0.8143
Epoch 7/10
337/337 [==============================] - 212s 630ms/step - loss: 0.3899 - accuracy: 0.8822 - val_loss: 1.0099 -

In [29]:
score = model.evaluate(X_zero_13_test, y_zero_13_test, verbose=0)
print('Test loss 13-zero:', score[0])
print('Test accuracy 13-zero:', score[1])

Test loss 13-zero: 0.8006333708763123
Test accuracy 13-zero: 0.790939450263977


In [31]:
score = model.evaluate(X_zero_22_test, y_zero_22_test, verbose=0)
print('Test loss 22-zero:', score[0])
print('Test accuracy 22-zero:', score[1])

Test loss 22-zero: 1.1063123941421509
Test accuracy 22-zero: 0.8102487921714783


In [22]:
score = model.evaluate(X_zero_31_test, y_zero_31_test, verbose=0)
print('Test loss 31-zero:', score[0])
print('Test accuracy 31-zero:', score[1])

Test loss 31-zero: 2.6865217685699463
Test accuracy 31-zero: 0.8236168026924133


In [15]:
score = model.evaluate(X_zero_40_test, y_zero_40_test, verbose=0)
print('Test loss 40-zero:', score[0])
print('Test accuracy 40-zero:', score[1])

Test loss 40-zero: 1.9793481826782227
Test accuracy 40-zero: 0.847382128238678


In [30]:
score = model.evaluate(X_time_13_test, y_time_13_test, verbose=0)
print('Test loss 13-time:', score[0])
print('Test accuracy 13-time:', score[1])

Test loss 13-time: 0.7413438558578491
Test accuracy 13-time: 0.8213887810707092


In [30]:
score = model.evaluate(X_time_22_test, y_time_22_test, verbose=0)
print('Test loss 22-time:', score[0])
print('Test accuracy 22-time:', score[1])

Test loss 22-time: 1.0273009538650513
Test accuracy 22-time: 0.826587438583374


In [20]:
score = model.evaluate(X_time_31_test, y_time_31_test, verbose=0)
print('Test loss 31-time:', score[0])
print('Test accuracy 31-time:', score[1])

Test loss 31-time: 1.3583720922470093
Test accuracy 31-time: 0.8069068193435669


In [25]:
score = model.evaluate(X_time_40_test, y_time_40_test, verbose=0)
print('Test loss 40-time:', score[0])
print('Test accuracy 40-time:', score[1])

Test loss 40-time: 1.5009771585464478
Test accuracy 40-time: 0.8057928085327148
